In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
maskf = xr.open_dataset("./LOCA2_MASKS.nc")
treedf = xr.open_dataset("http://kyrill.ias.sdsmt.edu:8080/thredds/dodsC/LOCA2/Original_CONUS/historical/LOCA2-CONUS___ACCESS-CM2.r1i1p1f1___historical.nc")

In [ ]:
huc08 = pd.read_excel(io         = "/Users/wjc/Documents/ArcGIS/Projects/LOCA2_Domain_Prep/WBD_HUC_Names.xlsx",
                      sheet_name = "WBDHU08").drop(columns=["huc08_areaacres"])

huc06 = pd.read_excel(io         = "/Users/wjc/Documents/ArcGIS/Projects/LOCA2_Domain_Prep/WBD_HUC_Names.xlsx",
                      sheet_name = "WBDHU06").drop(columns=["huc02",
                                                            "huc04",
                                                            "huc06_states",
                                                            "huc06_areaacres",
                                                            "huc06_areasqkm"]).set_index(["huc06"])
huc04 = pd.read_excel(io         = "/Users/wjc/Documents/ArcGIS/Projects/LOCA2_Domain_Prep/WBD_HUC_Names.xlsx",
                      sheet_name = "WBDHU04").drop(columns=["huc02",
                                                            "huc04_states",
                                                            "huc04_areaacres",
                                                            "huc04_areasqkm"]).set_index(["huc04"])
huc02 = pd.read_excel(io         = "/Users/wjc/Documents/ArcGIS/Projects/LOCA2_Domain_Prep/WBD_HUC_Names.xlsx",
                      sheet_name = "WBDHU02").drop(columns=["huc02_areaacres",
                                                            "huc02_states",
                                                            "huc02_areasqkm"]).set_index(["huc02"])

huc08 = huc08.set_index(["huc06"]).join(other = huc06)

huc08 = huc08.set_index(["huc04"]).join(other = huc04)

huc08 = huc08.set_index(["huc02"]).join(other = huc02)

huc08.reset_index(inplace = True)

huc08.drop(columns="huc02",
           inplace = True)

huc08 = huc08[["huc08",
               "huc08_name",
               "huc06_name",
               "huc04_name",
               "huc02_name",
               "huc08_states",
               "huc08_areasqkm"]].set_index(keys="huc08")

huc08.to_csv(path_or_buf = "./HUC08_LUT.csv")

huc08 = xr.Dataset.from_dataframe(huc08)

huc08["huc08"] = huc08["huc08"].astype(dtype = np.int32)

display(huc08)

In [ ]:
clim_divs = pd.read_excel(io         = "/Users/wjc/Documents/ArcGIS/Projects/LOCA2_Domain_Prep/NCEI_ClimDivs.xlsx")

clim_divs.rename(columns = {"CLIMDIV":    "climdiv", 
                            "FIPS_CD":    "climdiv_fips",
                            "STATE_FIPS": "climdiv_state_fips",
                            "CD_2DIG":    "climdiv_cd_2digit",
                            "CD_NEW":     "climdiv_cd_new",
                            "STATE_CODE": "climdiv_state_2digit",
                            "NAME":       "climdiv_name",
                            "STATE":      "climdiv_st_name",
                            "ST_ABBRV":   "climdiv_st_abbrv"},
                inplace = True)

clim_divs.set_index(keys = "climdiv", inplace = True)
clim_divs = xr.Dataset.from_dataframe(clim_divs)



clim_divs["climdiv"]              = clim_divs["climdiv"             ].astype(dtype = np.int32)
clim_divs["climdiv_fips"]         = clim_divs["climdiv_fips"        ].astype(dtype = np.int32)
clim_divs["climdiv_state_fips"]   = clim_divs["climdiv_state_fips"  ].astype(dtype = np.int32)
clim_divs["climdiv_state_2digit"] = clim_divs["climdiv_state_2digit"].astype(dtype = np.int32)
clim_divs["climdiv_cd_2digit"]    = clim_divs["climdiv_cd_2digit"   ].astype(dtype = np.int32)
clim_divs["climdiv_cd_new"]       = clim_divs["climdiv_cd_new"      ].astype(dtype = np.int32)
clim_divs



In [ ]:
maskf["lon"] = treedf["lon"]
maskf["lat"] = treedf["lat"]
maskf["lon_bnds"] = treedf["lon_bnds"]
maskf["lat_bnds"] = treedf["lat_bnds"]

In [ ]:
maskf["LOCA2_HUC08"].values = maskf["LOCA2_HUC08"].values  * maskf["LOCA2_MASK"].values
maskf["LOCA2_CLIMDIV"].values = maskf["LOCA2_CLIMDIV"].values  * maskf["LOCA2_MASK"].values


mask = maskf["LOCA2_MASK"]
hucs = maskf["LOCA2_HUC08"]
cds  = maskf["LOCA2_CLIMDIV"]



maskf["lon"]      = treedf["lon"]
maskf["lat"]      = treedf["lat"]
maskf["lon_bnds"] = treedf["lon_bnds"]
maskf["lat_bnds"] = treedf["lat_bnds"]

In [ ]:


maskf = maskf.merge(other = huc08)
maskf = maskf.merge(other = clim_divs)


In [ ]:
maskf.to_netcdf("./LOCA2_MASKS2.nc")